In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
%matplotlib inline
from pathlib import Path
import itertools

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams 

import seaborn as sns

from palmerpenguins import load_penguins

from matplottoy.data import iris
from matplottoy.encoders import color, position
from matplottoy.artists import point, bar, utils


spath = Path("../../../draft/figures/code/")
rcParams['figure.dpi'] = 200

In [ ]:
penguins = load_penguins()

In [ ]:
dict(penguins.dtypes)

In [ ]:
penguins['species'].value_counts().reset_index()

In [ ]:
table = iris.DataFrame(penguins['species'].value_counts().reset_index())

In [ ]:
table.view().keys()

In [ ]:
fts = {'position':{'name':'index', 
                   'encoder': position.Nominal({'Adelie': 0,  'Chinstrap': 1, 'Gentoo': 2})},  
        'length': {'name': 'species'},
        'facecolors': {'name':'index', 
                       'encoder': color.Categorical({'Adelie':'#FF8C00',
                                                'Gentoo':'#159090',
                                                'Chinstrap':'#A034F0'})}}

In [ ]:
table.view()['index']

In [ ]:
fig, ax = plt.subplots()

artistv = bar.BarArtist(table, fts, orientation='v')

In [45]:
%debug
fig, ax = plt.subplots()
artistv = bar.BarArtist(table, fts, orientation='v')
ax.add_artist(artistv)

[autoreload of matplottoy.artists.bar failed: Traceback (most recent call last):
  File "C:\Users\story\Anaconda3\envs\mplrelease\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\story\Anaconda3\envs\mplrelease\lib\site-packages\IPython\extensions\autoreload.py", line 394, in superreload
    module = reload(module)
  File "C:\Users\story\Anaconda3\envs\mplrelease\lib\imp.py", line 314, in reload
    return importlib.reload(module)
  File "C:\Users\story\Anaconda3\envs\mplrelease\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 779, in exec_module
  File "<frozen importlib._bootstrap_external>", line 916, in get_code
  File "<frozen importlib._bootstrap_external>", line 846, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_fra

> c:\users\story\projects\proposal\code\matplottoy\artists\bar.py(86)draw()
     84 
     85         for params in zip(visual.values()):
---> 86             vv = dict(zip(visual.keys(), params))
     87             print(vv)
     88             graphic = self.assemble(**))    



In [ ]:
%debug
fig, ax = plt.subplots()

artistv = bar.BarArtist(table, fts)
ax.add_artist(artistv)

# grouped bar

In [ ]:
penguins

In [ ]:
df = penguins.groupby(['island','species']).count()['sex'].reset_index()

In [ ]:
sb = penguins.groupby(['species', 'sex']).count()['year'].reset_index()

In [ ]:
sb.pivot(index=['sex'], columns=['species'], values = 'year').reset_index().plot.bar(stacked=False)

In [ ]:
sb.pivot(index=['sex'], columns=['species'], values = 'year').reset_index().plot.bar(stacked=True)

In [ ]:
sb

In [ ]:
wide_table = sb.pivot(index=['sex'], columns=['species'], values='year').reset_index()
for s in ['Adelie', 'Chinstrap', 'Gentoo']:
    wide_table[f'{s}_c'] = s

In [ ]:
bt = iris.DataFrame(wide_table)

In [ ]:
print(bt.view().to_latex())

In [ ]:
cmap = color.Categorical({'Adelie':'#FF8C00', 'Gentoo':'#159090','Chinstrap':'#A034F0'})
ts = {'position': {'name':'sex',
                    'encoder': position.Nominal({'female':0, 'male':1})}} 
mgts = {'length': {'name': ['Adelie', 'Chinstrap', 'Gentoo']},
        'facecolors': {'name': ['Adelie', 'Chinstrap', 'Gentoo'], 
                       'encoder': cmap}} 
sgts = {'length':{'name': ['Adelie']},
        'facecolors' :{'name':['Adelie'], 'encoder':cmap}}

In [ ]:
group_transforms =  [{'length': {'name':'Adelie'},
                      'facecolors': {'name':"Adelie_c", 'encoder':cmap}},
                     {'length': {'name':'Chinstrap'},
                      'facecolors': {'name':"Chinstrap_c", 'encoder':cmap}}, 
                     {'length': {'name':'Gentoo'},
                      'facecolors': {'name':"Gentoo_c", 'encoder':cmap}}]

In [ ]:
# to do - this should just work
should_do = {'position': {'name':'sex',  'encoder': position.Nominal({'female':0, 'male':1})}, 
      'length': {'name': 'year'},
      'facecolors':{'name': 'species', 'encoder': cmap}}

In [ ]:
fig, ax = plt.subplots()
artistSB = bar.StackedBar(bt, ts, group_transforms)
ax.add_artist(artistSB)
_ = ax.set(ylim = (0, 175), 
       xlim = (-.2, 2),  
        xticks = (.4, 1.4),
        xticklabels = ['female', 'male'],
       xlabel = "")
ax.set_ylabel("# penguins", fontsize=14 )
ax.tick_params(labelsize=12)
fig.savefig(spath/"bar_stacked.png",  bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots()
artistGB = bar.GroupedBar(bt, ts, group_transforms)
ax.add_artist(artistGB)
_ = ax.set(ylim = (0, 80), 
       xlim = (-.2, 2),  
            xticks = (.4, 1.4),
        xticklabels = ['female', 'male'],
       xlabel = "")
ax.set_ylabel("# penguins", fontsize=14 )
ax.tick_params(labelsize=12)
fig.savefig(spath/"bar_grouped.png",  bbox_inches='tight')

In [ ]:
%matplotlib inline
import matplotlib.path as mpath
import matplotlib.patches as mpatches

fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, constrained_layout=True)
for a in artistv._paths:
    path = mpath.Path(a.vertices, a.codes, closed=True)
    patch = mpatches.PathPatch(path)
    ax1.add_patch(patch)
_ = ax1.set(ylim = (0, 160), xlim = (-0.25, 3))
for c, col in zip(artistSB.children, ['C0', 'C1', 'C2']):
    for a in c._paths:
        path = mpath.Path(a.vertices, a.codes, closed=True)
        patch = mpatches.PathPatch(path, color=col)
        ax2.add_patch(patch)
_ = ax2.set(ylim = (0, 170), xlim = (-0, 3))
for c, col in zip(artistGB.children, ['C0', 'C1', 'C2']):
    for a in c._paths:
        path = mpath.Path(a.vertices, a.codes, closed=True)
        patch = mpatches.PathPatch(path, color=col)
        ax3.add_patch(patch)
_ = ax3.set(ylim = (0, 170), xlim = (-0, 3))